In [1]:
# Dependencies
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K 
import os
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
from IPython.display import display
import random
import itertools as itt

img_width, img_height = 224, 224

In [2]:
def build_model():
    model = ResNet50(
        include_top=False,
        weights="imagenet",
        input_shape=(224, 224, 3),
        pooling='avg',
        classes=2
    )
    
    for layer in model.layers:
        layer.trainable = False
    
    output = Dense(2,activation='tanh')(model.output)
    model = Model(model.input,output)
    
    model.compile(optimizer='adam', loss='mse')
    
    return model


In [3]:
model = build_model()
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [4]:
def load_dataset(folders):
    paths = []
    inputs = []
    outputs = []

    for tag,folder in folders:
        for filename in os.listdir(folder):
            img_path = folder + filename
            paths.append(img_path)
            img = image.load_img(img_path, target_size=(224, 224))
            input_arr = image.img_to_array(img)
            inputs.append(input_arr)
            output_arr = [tag == 'cat', tag == 'dog']
            outputs.append(output_arr)

    inputs = preprocess_input(np.array(inputs))
    outputs = np.array(outputs).astype(np.float32)
    
    return paths,inputs,outputs

In [8]:
def load_training_dataset():
    training_folders = [
        ('cat','input/dogs-cats-images/dataset/training_set/cats/'),
        ('dog','input/dogs-cats-images/dataset/training_set/dogs/')
    ]
    
    return load_dataset(training_folders)[1:]


In [9]:
def shuffle_data(inputs,outputs):
    max_len = min(len(inputs),len(outputs))
    indexes = list(enumerate(np.random.randn(max_len)))
    indexes.sort(key=lambda x:x[1])
    indexes = list(map(lambda x:x[0],indexes))
    
    return inputs[indexes],outputs[indexes]


In [10]:
train_X, train_y = load_training_dataset()
train_X, train_y = shuffle_data(train_X, train_y)

In [11]:
model.fit(train_X, train_y,
          batch_size=64, epochs=20)

Epoch 1/20
  9/125 [=>............................] - ETA: 7:13 - loss: 0.3785

KeyboardInterrupt: 

In [ ]:
model.save('model_cat_dog.h5') 

In [ ]:
# Evaluate the model using the training data
model_loss, model_accuracy = model.evaluate(validation_generator, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Load the model
from tensorflow.keras.models import load_model
model_path = 'C:/Users/tgokh/Documents/Final-Project/model_cat_dog.h5'
modelx = load_model(model_path)

In [ ]:
from keras.models import load_model
import cv2
import numpy as np

model = load_model('model_cat_dog.h5')

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

img = cv2.imread('input/dogs-cats-images/dataset/test_set/dogs/dog.4501.jpg')
img = cv2.resize(img,(224, 224))
img = np.reshape(img,[1,224, 224,3])

classes = model.predict_classes(img)

print(classes)

In [ ]:
classes = train_generator.class_indices    
print(classes)

In [ ]:
from IPython.display import display, Image
display(Image(filename='input/dogs-cats-images/dataset/test_set/dogs/dog.4501.jpg'))

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))